In [17]:
import pandas as pd
import matplotlib.pyplot as plt

In [21]:
full_file = pd.read_csv("albumlist.csv", encoding='latin-1')
full_file.head(1)

,Number,Year,Album,Artist,Genre,Subgenre
0,1,1967,Sgt. Pepper's Lonely Hearts Club Band,The Beatles,Rock,"Rock & Roll, Psychedelic Rock"


In [23]:
grouped_by_Genre= full_file.groupby(["Genre", "Year"]) ["Artist"].count()
grouped_by_Genre

Genre                 Year
Blues                 1958    1
                      1962    1
                      1964    1
                      1965    1
                      1967    1
                             ..
Rock, Stage & Screen  1964    1
                      1965    1
                      1970    1
Rock,ÊBlues           1966    1
Rock,ÊPop             1990    1
Name: Artist, Length: 252, dtype: int64

In [20]:
print ("Hello world!")

Hello world!
